# Objectif

Proposer un affichage de résultats (sous forme de pages HTML articulées entre elles) à partir du jeu de données (4 fichiers Unimarc XML d'entités LRM) et des objets Python créés dessus, à partir du code présenté dans le notebook **Manipulation des fichiers de notices Unimarc LRM**

Ce code sera amené à évoluer selon les besoins des affichages

Le coeur du dispositif est un dictionnaire d'instances de classes Item, Manif, Expression, Oeuvre et Record (pour les autres notices : personnes et sujets notamment) dont la clé est le numéro de notice, et la valeur l'instance en question.

In [1]:
import os
from Record import construct_indexation
from generate_records import generate_dict_entities
from generate_results import search, display_html_results, delete_html_results

### Préalable : génération du dictionnaire d'entités

In [2]:
dict_entities = generate_dict_entities("UMA_Oeuvres_Expressions.xml", "UMB_Manifestations.xml",
                                       "UMH_Items.xml", "UMA_Autres_Entites_Liees.xml")
oeuvres = [dict_entities[o] for o in dict_entities if dict_entities[o].type == "o"]
expressions = [dict_entities[e] for e in dict_entities if dict_entities[e].type == "e"]

# Moteur de recherche (afficher des oeuvres)

In [ ]:
kw_search = input("Chercher dans la base : ")
results_o = search(kw_search, oeuvres, dict_entities, "all")
results_e = search(kw_search, expressions, dict_entities, "all")

print("Nombre de résultats :", len(results_o), "oeuvre(s) - ", len(results_e), "expression(s)")
display_html_results(results_o, dict_entities, kw_search, "o")
display_html_results(results_e, dict_entities, kw_search, "e")

**[Résultats par oeuvres](results/short_results_o.html)** **[Résultats par expressions](results/short_results_e.html)**

### Supprimer les pages de résultats

Lancer la cellule ci-dessous

In [9]:
delete_html_results()

# Zipper les résultats

In [4]:
from zipfile import ZipFile
import os

In [5]:
results = ZipFile("results.zip", "w")
for file in os.listdir("results"):
    results.write(os.path.join("results", file))
subdirs = ["results/graphs", "results/icons"]
for subdir in subdirs:
    for file in os.listdir(subdir):
        results.write(os.path.join(subdir, file))
    

In [6]:
results.close()

# Génération d'une arborescence des résultats

Générer une page HTML qui contienne des listes imbriquées : 
* pour chaque oeuvre : la liste des expressions
* pour chaque expression : la liste des manifestations
* pour chaque manifestation : la liste des items

In [19]:
listes_file = open("liste_entites.html", "w", encoding="utf-8")
listes_file.write("<html>\n<meta encoding='utf-8'/>\n<head>\n<title>Liste des entités</title>\n\
<style type='text/css'>a {text-decoration: none;}\n\
body > ol > li {margin-top: 20px;}\n\
body > ol > li > ol > li {margin-top: 5px;}</style></head>\n")
listes_file.write("<body>\n<h1>Liste des entités</h1>\n")
listes_file.write("<ol>")

4

In [20]:
for o in oeuvres:
    listes_file.write(f"<li><strong><a href='results/full_results_{o.id}.html'>{o.label}</a></strong>")
    if len(dict_entities[o.id].toExpressions):
        listes_file.write("\n<ol>")
        for eid in dict_entities[o.id].toExpressions:
            listes_file.write(f"\n<li><a href='results/full_results_{eid}.html'>{dict_entities[o.id].toExpressions[eid]}</a></li>")
            manifs = dict_entities[eid].toManifs
            if len(manifs):
                listes_file.write("\n<ul>")
                for manifid in manifs:
                    listes_file.write(f"\n<li>{dict_entities[manifid].label}</li>")
                listes_file.write("\n</ul>")
        listes_file.write("\n</ol>")
    listes_file.write("\n</li>\n")
listes_file.write("</ol>\n</body>\n</html>")
listes_file.close()